In [1]:
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig
import soundfile as sf
import torch

/Users/Benjamin/dev/ssa/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_path = "microsoft/Phi-4-multimodal-instruct"
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="mps",
    _attn_implementation="eager",  # Correct way to disable Flash Attention
)

AttributeError: 'Phi4MMModel' object has no attribute 'prepare_inputs_for_generation'

In [8]:
# After loading the model
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-multimodal-instruct",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="mps",
    _attn_implementation="eager",
)


# Patch the missing method
def prepare_inputs_for_generation(self, *args, **kwargs):
    return args, kwargs


# Attach to the relevant submodule (may be model.model or model.base_model)
setattr(
    model.base_model,
    "prepare_inputs_for_generation",
    prepare_inputs_for_generation.__get__(model.base_model, type(model.base_model)),
)

AttributeError: 'Phi4MMModel' object has no attribute 'prepare_inputs_for_generation'

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-multimodal-instruct",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="mps",
    _attn_implementation="eager",
)


def add_prepare_inputs_for_generation(obj):
    def prepare_inputs_for_generation(self, *args, **kwargs):
        return args, kwargs

    setattr(
        obj,
        "prepare_inputs_for_generation",
        prepare_inputs_for_generation.__get__(obj, type(obj)),
    )


try:
    add_prepare_inputs_for_generation(model.base_model)
except Exception:
    pass
try:
    add_prepare_inputs_for_generation(model.base_model.model)
except Exception:
    pass
try:
    add_prepare_inputs_for_generation(model.model)
except Exception:
    pass
try:
    add_prepare_inputs_for_generation(model)
except Exception:
    pass


AttributeError: 'Phi4MMModel' object has no attribute 'prepare_inputs_for_generation'

In [14]:
# import sys

# modeling_module_name = "modeling_phi4mm"

# if modeling_module_name in sys.modules:
#     module = sys.modules[modeling_module_name]
# else:
#     raise ImportError("Custom model module not found in sys.modules.")

# if hasattr(module, "Phi4MMModel"):
#     Phi4MMModel = getattr(module, "Phi4MMModel")

#     def prepare_inputs_for_generation(self, *args, **kwargs):
#         # Dummy implementation for compatibility
#         return args, kwargs

#     setattr(Phi4MMModel, "prepare_inputs_for_generation", prepare_inputs_for_generation)

from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-multimodal-instruct",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="mps",
    _attn_implementation="eager",
)


AttributeError: 'Phi4MMModel' object has no attribute 'prepare_inputs_for_generation'

In [15]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="microsoft/Phi-4-multimodal-instruct",
    local_dir="/Users/Benjamin/dev/ssa/models/Phi-4-multimodal-instruct",
)

Fetching 54 files: 100%|██████████| 54/54 [00:29<00:00,  1.83it/s]


'/Users/Benjamin/dev/ssa/models/Phi-4-multimodal-instruct'

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM, GenerationConfig
import soundfile as sf
import torch

# had to copy and paste the prepare_inputs_for_generation method from /Users/Benjamin/dev/ssa/.venv/lib/python3.12/site-packages/transformers/generation/utils.py into the Phi4MMModel class
model_path = "/Users/Benjamin/dev/ssa/models/Phi-4-multimodal-instruct"
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="mps",
    _attn_implementation="eager",  # Correct way to disable Flash Attention
)

/Users/Benjamin/dev/ssa/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/Benjamin/dev/ssa/.venv/lib/python3.12/site-packages/transformers/models/auto/image_processing_auto.py:625: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Phi4MMModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However

In [8]:
audio_data, sample_rate = sf.read(
    "/Users/Benjamin/dev/ssa/data/fluencybank/processed/wav_clips/20f_002_000.wav"
)

In [ ]:
user_prompt = "<|user|>"
assistant_prompt = "<|assistant|>"
prompt_suffix = "<|end|>"

speech_prompt = "Based on the attached audio, transcribe the spoken content."
prompt = f"{user_prompt}<|audio_1|>{speech_prompt}{prompt_suffix}{assistant_prompt}"

# Prepare input
inputs = processor(
    text=prompt, audios=[(audio_data, sample_rate)], return_tensors="pt"
).to("mps")

# Run generation
generation_config = GenerationConfig.from_pretrained(model_path)
generate_ids = model.generate(
    **inputs,
    max_new_tokens=1200,
    generation_config=generation_config,
)
# Remove prompt tokens from output
output_ids = generate_ids[:, inputs["input_ids"].shape[1] :]
response = processor.batch_decode(
    output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]
print(response)


/Users/Benjamin/dev/ssa/.venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


TypeError: bad operand type for unary -: 'NoneType'